# Variant calling and genotype calling

let's continue from yesterdays exercises where we mapped data. 

## Setup environment

First we need to go to the same folder and make sure 
we have all the files and software we need. 

In [ ]:
COURSE_PATH=/course/popgen25
DATA_PATH=$COURSE_PATH/NGSInference
SOFTWARE_PATH=$COURSE_PATH/software

REF=${DATA_PATH}/data/ref.fa.gz
ANC=${DATA_PATH}/data/anc.fa.gz


echo --programs that are installed:--

which samtools
which bcftools
which angsd

# enter folder
mkdir -p ~/popgen25_NGSinference
cd ~/popgen25_NGSinference

#make sym link for data and current folder
ln -sfn ~/popgen25_NGSinference ~/current_folder
ln -sfn $DATA_PATH ~/data_folder

echo --Reference fasta for goat:--
ls ~/data_folder/fasta/goat.fa.gz*


We will use the SAM (BAM) files which has been generated during the exercise yesterday.

In case that you cannot find them, we also prepared the BAM file for you.

In [ ]:
# this exercise depends on bam file made yesterday - folder for this 
ls ~/popgen25_ngsintro/CTauTzS_8872.sorted.bam
if [ -f ~/popgen25_ngsintro/CTauTzS_8872.sorted.bam ]; then ln -sfn ~/popgen25_ngsintro/CTauTzS_8872.sorted.bam ./; fi


In [ ]:

# If it says "No such file or directory", then let's use this pre-generated file
ls ${DATA_PATH}/sams/CTauTzS_8872.sorted.bam
if [ -f ${DATA_PATH}/sams/CTauTzS_8872.sorted.bam ]; then ln -sfn ${DATA_PATH}/sams/CTauTzS_8872.sorted.bam ./; fi


### create VCF file
Lets create a VCF file for the CTauTzS_8872.sorted.bam. This is done using bcftools. The ploidy is diploid (2) for mammals but otherwise we use the default settings


In [ ]:
## create index for the BAM file
samtools index CTauTzS_8872.sorted.bam

## call variants
bcftools mpileup -Ou -f ~/data_folder/fasta/goat.fa.gz CTauTzS_8872.sorted.bam -r NC_030808.1:1-1000000 | bcftools call --ploidy 2 -mv -a GQ  -Ov -o CTauTzS_8872.vcf


Lets have a look at the VCF file


In [ ]:
head -n 100 CTauTzS_8872.vcf


 The header of the VCF contains meta information about what it in the file.
In the body of the file
 - Identify the position, the reference allele and the alternative allele of the file.
 - Identify the depth of each position
 - Identify the genotype quality for each genotype call
 
 There are many sites with too little information to call variants. 
 
 Let's apply some light filters. Here we remove sites with less than 8 reads (DP<8) and sites with a low quality score (QUAL<20)


In [ ]:

bcftools filter CTauTzS_8872.vcf -e 'QUAL<20 || DP < 8' > CTauTzS_8872.filt.vcf

head -n 100 CTauTzS_8872.filt.vcf



 - How many sites are are called as variable?
 - Find a heterozygous site.  ( look for 0/1)





 Look at the alignment for the position
 

In [ ]:

POS=6833

#the site you choose is the first base of the alignment so we center it by subtracting 50 to the position
POS50=$(($POS  - 50 ))
samtools tview  CTauTzS_8872.sorted.bam  -d T -w 100 -p NC_030808.1:$POS50


We can use the mpileup option to get a summary of the data at that position

In [ ]:

echo -e "CHR\tPOS\tREF\tDEPTH\tBASES\tbaseQuality" > mpileup.file
samtools mpileup CTauTzS_8872.sorted.bam -r NC_030808.1:$POS-$POS >> mpileup.file

echo --- pileup of the site which shows the bases and their score ---
column -t mpileup.file


 - At that position count the number of bases of different types
   - #A = ??
   - #C = ??
   - #G = ??
   - #T = ??

 - Calculate the genotype likelihoods and call the genotype for the site. 
 
You can use the shiny app to help with the calculations for calculating genotype likelihoods using the GATK model. Enter the BASES and their base qualities. Keep the prior (allele frequency) field blank.

https://popgen.dk/shiny/anders/GL/


The resulting values will not be exactly the same as in bcftools since bcftools uses a slightly different way of calculating the genotype likelihoods than the GATK model. 


# Exercises in low depth NGS data

Exercises is a modified version of [Matteo Fumagilli's exercises] that i previously did for this summer course(https://github.com/mfumagalli/Copenhagen)

In this session you will learn how to do:
* genotype calling
* allele frequency estimation
* variant (or SNP) calling
### Data

As an illustration, we will use 40 BAM files of human samples (of African, European, East Asian, and Native American descent), a reference genome, and putative ancestral sequence. We will also use 10 BAM files of Latinos in one example.


*MOTIVATION*
Detecting signatures of natural selection in the genome has the twofold meaning of (i) understanding which adaptive processes shaped genetic variation and (ii) identifying putative functional variants.
In case of humans, biological pathways enriched with selection signatures include pigmentation, immune-system regulation and metabolic processes.
The latter may be related to human adaptation to different diet regimes, depending on local food availability (e.g. the case of lactase persistence in dairy-practicing populations).

The human Ectodysplasin A receptor gene, or EDAR, is part of the EDA signalling pathway which specifies prenatally the location, size and shape of ectodermal appendages (such as hair follicles, teeth and glands).
EDAR is a textbook example of positive selection in East Asians (Sabeti et al. 2007 Nature) with tested phenotypic effects (using transgenic mice).

Recently, a genome-wide association study found the same functional variant in EDAR associated to several human facial traits (ear shape, chin protusion, ...) in Native American populations (Adhikari et al. Nat Commun 2016).

![](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference/practical.png)


*HYPOTHESIS*

- Is the functional allele in East Asian at high frequency in other human populations (e.g. Native Americans)?
- Can we identify signatures of natural selection on EDAR in Native Americans?
- Is selection targeting the same functional variant?

*CHALLENGES*
- Admixed population
- Low-depth sequencing data
- Effect of genetic drift
- ...



We are using the program [ANGSD](http://popgen.dk/wiki/index.php/ANGSD) (Analysis of Next Generation Sequencing Data).
More information about its rationale and implemented methods can be found [here](http://www.ncbi.nlm.nih.gov/pubmed/25420514).

According to its website *ANGSD is a software for analyzing next generation sequencing data. The software can handle a number of different input types from mapped reads to imputed genotype probabilities. Most methods take genotype uncertainty into account instead of basing the analysis on called genotypes. This is especially useful for low and medium depth data.*

Please make sure to follow these preparatory instructions below before running these examples. 
Briefly, you need to first login to the server with your user name and password. And once you are logged in you should set the path to the software and various data that will be used by pasting in the following code in the terminal:




In [ ]:
#REF=${DATA_PATH}/data/ref.fa.gz
#ANC=${DATA_PATH}/data/anc.fa.gz

## see paths of used programs
which angsd
which samtools

## check the reference sequences we need
ls ${REF}
ls ${ANC}



Also, you will have to create two folders in your working directory, one for your results and one for your intermediate data, which you can do by running the following code:


In [ ]:
# make sure we're in the working space
cd ~/current_folder/

mkdir -p Results
mkdir -p Data

ls

### Workflow

The workflow for this practical looks like this

![stages](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference/stages.png)

which seems daunting! 
However, that's actually not the case and we will go through each step to understand each one of them.

The workflow is roughty divided into four steps:

0. Data filtering and I/O
1. Genotype likelihoods
2. Genotype calling
3. SNP calling



#### 0. Data filtering and I/O

First, we will learn **how to build a command line in ANGSD**.

![stage0](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage0.png)

To see a full list of options in ANGSD type the following in the terminal:


In [ ]:
angsd

ANGSD can accept several input files, as described [here](http://popgen.dk/angsd/index.php/Input):

* BAM, CRAM, mpileup
* VCF, GLF, beagle

Here we show how ANGSD can also perform some basic filtering of the data.
These filters are based on:

* quality and depth, see [here](http://www.popgen.dk/angsd/index.php/Filters)
* SNP quality, see [here](http://popgen.dk/angsd/index.php/SnpFilters)
* sites, see [here](http://popgen.dk/angsd/index.php/Sites)

### Make list of bam files
In our data we have alignment files (BAM files) from four populations. Let's make file list with the BAM files for each population seperately and one list with all samples by running the following code 


In [ ]:
ls ${DATA_PATH}/data/CHB.BAMs/*bam > EAS.bams
ls ${DATA_PATH}/data/TSI.BAMs/*bam > EUR.bams
ls ${DATA_PATH}/data/NAM.BAMs/*bam > NAM.bams
ls ${DATA_PATH}/data/PEL.BAMs/*bam > LAT.bams
ls ${DATA_PATH}/data/LWK.BAMs/*bam > AFR.bams

cat EAS.bams EUR.bams NAM.bams LAT.bams AFR.bams > ALL.bams

have a look at our list of BAM files



In [ ]:

## print path of all bam files
cat ALL.bams
echo \# count number of bam files
wc -l ALL.bams
echo \# show the bam file lists
ls *.bams



If the input file is in BAM format, the possible options are visible if you run the command `angsd -bam`.


In [ ]:
angsd -bam

First we need to define input and output files (please note that here we do not run these intermediate steps, as you can see thare is a ```#``` in the front):
```bash
angsd -b ALL.bams -ref $REF -out Results/ALL \
```


with `-b` we give the file including paths to all BAM files we need to analyse.
`-ref` specifies the reference sequence.
`-out` states the prefix for all output files that will be generated.

Next we need to define some basic filtering options.
First we define filters based on reads quality.


```bash
 angsd -b ALL.bams -ref $REF -out Results/ALL \
        -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
```



These filters will retain only uniquely mapping reads, not tagged as bad, considering only proper pairs, without trimming, and adjusting for indel/mapping (as in samtools).
`-C 50` reduces the effect of reads with excessive mismatches, while `-baq 1` computes base alignment quality as explained here ([BAQ](http://samtools.sourceforge.net/mpileup.shtml)) used to rule out false SNPs close to INDELS.

Also, you may want to remove reads with low mapping quality and sites with low quality or covered by few reads (low depth).
Under these circumstances, the assignment of individual genotypes and SNPs is problematic, and can lead to errors.
We may also want to remove sites where a fraction (half?) of the individuals have no data.
This is achieved by the ```-minInd``` option.

In order to understand which filters to use, it is important to visualise the distribution of quality scores and depth. Run the following command


In [ ]:
angsd  -b EUR.bams  -ref $REF -out Results/EUR \
        -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
        -minMapQ 20 \
        -doQsDist 1 -doDepth 1 -doCounts 1 -maxDepth 70

Angsd will give a warning that the BAM files do not contain data for the whole genome. 

- make a plot of the results


In [ ]:
# quick script to compute percentiles and plot distributions of quality scores and depths

fin <-"~/current_folder/Results/EUR"
bam.list <- ""
samples <- paste("sample",1:1e5)

par(mfrow=c(3,1))

## barplot q-scores
qs <- read.table(paste(fin,".qs",sep="",collapse=""), head=T, stringsAsFactors=F)
barplot(height=qs$counts, names.arg=qs$qscore, xlab="Q-score", ylab="Counts",main=basename(bam.list))
qs <- cbind(qs, perc=cumsum(qs$counts/1e4)/sum(qs$counts/1e4,na.rm=T))

## global depth
dep <- as.numeric(scan(paste(fin,".depthGlobal", sep="",collapse=""),what="char", quiet=T))
barplot(height=dep, names.arg=seq(1,length(dep))-1, xlab="Global Depth", ylab="Counts",main=basename(bam.list))

## sample depth
deps <- t(read.table(paste(fin,".depthSample", sep="",collapse=""),head=F, stringsAsFactors=F))
## per sample
max_x <- 2 * nrow(deps)/ncol(deps)
for (i in 1:ncol(deps)) {
  barplot(height=deps[1:max_x,i], names.arg=seq(1,max_x)-1, xlab="Sample Depth", ylab="Counts",main=samples[i])
}



![stage0A](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage0A.png)

Based on the plots (global depth) one could keep sites with a least 7 reads accross sample and with less than 50 reads. A possible command line would contain the following filtering:


```bash
angsd -b ALL.bams -ref $REF -out Results/ALL \
        -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
        -minMapQ 20 -minQ 20 -minInd 5 -setMinDepth 14 -setMaxDepth 50 -doCounts 1 
```


which corresponds to the following scenario:

Parameter | Meaning |
--- | --- |
-minInd 5 | use only sites with data from at least 5 individuals |
-setMinDepth 14 | minimum total depth of 14 |
-setMaxDepth 50 | maximum total depth of 50 |

More sophisticated filtering can be done, but this is outside the scope of this practical.

---------------------------------------

#### 1. Genotype likelihoods

![stage1](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference/stage1.png)

We now wish to calculate the ***genotype likelihoods*** for each site at each individual.

To do so you need to specify which genotype likelihood model to use.


In [ ]:
angsd -GL

A description of these different models can be found [here](http://www.popgen.dk/angsd/index.php/Genotype_likelihoods).
Briefly, the GATK model refers to the first GATK paper, SAMtools is somehow more sophisticated (non-independence of errors), SOAPsnp requires a reference sequence for recalibration of quality scores, SYK is error-type specific.
For most applications and data, GATK and SAMtools models should give similar results.

Let's assume to analyse European.
A possible command line to estimate allele frequencies can be seen below. Run the command ( takes ~1 min, "[*]" means it is still running)



In [ ]:
#run angsd command
angsd -b EUR.bams -ref $REF -out Results/EUR \
        -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
        -minMapQ 20 -minQ 20 -minInd 5 -setMinDepth 7 -setMaxDepth 30 -doCounts 1 \
        -GL 2 -doGlf 4
# No data for chromoId=0 chromoname=1

where we specify:
* -GL 2: genotype likelihood model as in GATK
* -doGlf 4: output in text format

Ignore the various warning messages (if any).

![stage1A](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage1A.png)

**QUESTION**
What are the output files?

<details>

<summary> click here to show the answer </summary>

```bash
The output from angsd tells you it made two files called "Results/EUR.arg" and "Results/EUR.glf.gz"
	.....
	-> Done reading data waiting for calculations to finish
	-> Done waiting for threads
	-> Output filenames:
		->"Results/EUR.arg"
		->"Results/EUR.glf.gz"
	-> Mon Aug  7 20:50:39 2023
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 983839
	-> Number of sites retained after filtering: 444928 
	[ALL done] cpu-time used =  57.66 sec
	[ALL done] walltime used =  57.00 sec

```
</details>

One files contains information of all the used options and the other contains the 10 genotype likelihoods (log) for each individual. 
View the file with

In [ ]:
#view first 1 lines
zcat Results/EUR.glf.gz  | head -n 1

A value of 0 means no reads or a high genotype likelihood ( GL=1 -> log(GL)~0). The first 2 columns denotes the chromosomes and position. The rest is 10 genotypes likelihoods for each of the individuals.  

lets look at a sites with data from individual one at a certain position together with the obversed bases

In [ ]:
#(ignore broken pipe)
echo -e "Chr\tPos\tAA\tAC\tAG\tAT\tCC\tCG\tCT\tGG\tGT\tTT" > res
zcat Results/EUR.glf.gz | cut -f 1-12 | head -n1250 | tail -n 1 >>res
echo -e "\n\n----10 GLs-----"
column -t res


echo -e "\n\n----mpileup for the same position-----"
echo -e "\nChr\tPos\tREF\tDepth\tObservedBases" > res
samtools mpileup -r 2:109002954-109002954 -b EUR.bams | cut -f 1-5 >> res
#print results
column -t res

# cut: write error: Broken pipe

#### 2. Genotype calling

![stage2](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage2.png)

Here we will explore several ways to call genotypes from sequencing data.
We will also calculate genotypes probabilities to each site for each individual.

In ANGSD, the option to call genotypes is `-doGeno`:

In [ ]:
angsd -doGeno

Therefore, if we set `-doGeno 2`, genotypes are coded as 0,1,2, as the number of alternate alleles. A value of -1 indicates a missing (uncalled) genotype.
If we want to print the major and minor alleles as well then we set `-doGeno 3`.

To calculate the posterior probability of genotypes we need to define a model.

In [ ]:
angsd -doPost

Furthermore, this calculation requires the specification of how to assign the major and minor alleles (if biallelic).

In [ ]:
angsd -doMajorMinor


We can call genotypes from the genotype likelihoods ( Results/EUR.glf.gz) by first identifying the major and minor alleles accross samples and the choosing the genotype with the highest genotype likelihood. 


In [ ]:
angsd -b EUR.bams -ref $REF -out Results/EUR \
        -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
        -minMapQ 20 -minQ 20 -minInd 5 -setMinDepth 7 -setMaxDepth 30 -doCounts 1 \
        -GL 2 -doGlf 1 


angsd -glf Results/EUR.glf.gz -fai $REF.fai -nInd 10 -out Results/EUR \
    -doMajorMinor 1 -doGeno 3 -doPost 2 -doMaf 1


Let's ignore the `-doMaf` option now. Note the number of sites retained after filtering

- how many sites in your data?
- For how many sites did you try call genotypes?

Have a look at the first 10 lines of the output file by running the follwoing command:


In [ ]:
#-> Total number of sites analyzed: 983839
#	-> Number of sites retained after filtering: 444928 

In [ ]:
zcat Results/EUR.geno.gz | head -n 10

The columns are: chromosome, position, major allele, minor allele, genotypes is 0,1,2 format.



The following code counts the number of sites with least one missing genotype? 


In [ ]:
zcat Results/EUR.geno.gz | grep -1 - | wc -l

- How many sites have no missing genotypes?  Why so few?

In [ ]:
zcat Results/EUR.geno.gz | grep -v -1 - | wc -l

<details>

<summary> click here to show the answer </summary>
    The mean depth per sample is around 2-3X, therefore genotypes cannot be assigned with very high confidence.
Setting this threshold depends on the mean sequencing depth of your data, as well as your application.
For some analyses you need to work only with high quality genotypes (e.g. measure of proportion of shared SNPs for gene flow estimate), while for others you can be more relaxed (e.g. estimate of overall nucleotide diversity).
We will show later how to accurately estimate summary statistics with low-depth data.


</details>


![stage2A](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage2A.png)
**EXERCISE 1**

If we calculate the allele frequency and assume hwe then we can use this information as prior probability to calculate genotype posterior probabilities.
The command line would be:

In [ ]:
angsd -glf Results/EUR.glf.gz -fai $REF.fai -nInd 10 -out Results/EUR \
        -doMajorMinor 1 -doGeno 3 -doPost 1 -doMaf 1

using the option `-doPost 1`.

We are finally ready to gather some biological insights from the data.
Recalling our research aim, our first goal is to calculate allele frequencies of our target variant in EDAR gene for different human populations.

Our SNP of interest is located in EDAR gene.
According to the reference [paper](http://www.nature.com/ncomms/2016/160519/ncomms11616/full/ncomms11616.html), *The derived G allele at the index SNP in this region (rs3827760) encodes a functional substitution in the intracellular death domain of EDAR (370A) and is associated with reduced chin protrusion*.
The genomic location of this SNP is `chr2:109513601-109513601`.

In ANGSD we can restrict our analyses on single region using the option `-r 2:109513601`

We are interested in calculating the derived allele frequencies, so are using the ancestral sequence to polarise the alleles.
We also want to compute the allele frequencies for each population separately.
We need to use a different file for each population, with a different list of BAM files, as provided:


In [ ]:
ls *.bams


We retain only these populations: AFR (Africans), EUR (Europeans), EAS (East Asians), LAT (Latinos), NAM (Native Americans).

Back to our example of functional variants in EDAR, we want to assign individual genotypes by first computing genotype posterior probabilities for all samples.
Finally we will calculate the **derived** allele frequencies based on assigned genotypes.

Note that we are interested in calculating the **derived** allele frequency, so we need to specify a putative ancestral sequence.
Let's assume that our reference sequence represents the ancestral sequence too.
Please finally note that we want to relax out filtering to make sure to have results.

Write the code that performs the following genotype calling for EDAR variants in all populations.
Also, you can directly call genotypes without generating the genotype likelihood files, by starting from BAM files directly.
As an indication, you can follow these guidelines:
- use the SAMtools genotype likelihood model
- calculate genotype posterior probabilities using a HWE-based prior
- filter out bases with a quality score less than 20
- filter our reads with a mapping quality score less than 20
- use ony sites where you have at least one sample with data (-mindInd)
- do not set any filtering based on min and max depth
- use -doMajorMinor 1 and -doMaf 1 options
- set genotypes as missing if the highest genotype probability is less than 0.50
- use option `-r 2:109513601` to restrict the analysis only to that site
but feel free to choose some parameters yourself.

Run the below code where we loop over each population to generate results for each of them. 



In [ ]:
for POP in AFR EUR EAS LAT NAM
do
        echo $POP
        angsd -b $POP.bams -ref $REF -anc $ANC -out Results/$POP.EDAR \
                -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
                -minMapQ 20 -minQ 20 -minInd 1 -setMinDepth 1 -setMaxDepth 100 -doCounts 1 \
                -GL 1 -doMajorMinor 5 -doMaf 1 -skipTriallelic 1 \
                -doGeno 3 -doPost 1 -postCutoff 0.50 \
                -r 2:109513601
done

You can view the genotype calls with       

In [ ]:
for POP in AFR EUR EAS LAT NAM
do
    echo $POP
    zcat Results/$POP.EDAR.geno.gz
done


For instance, you may have 0/20 in AFR and EUR, 20/20 in EAS, while there are only 4 called genotypes in NAM.




Once done, open the output files and calculate the derived allele frequency by counting genotypes.
What is the derived allele frequency for each population?

Can you comment these results?
Do you see any allele frequency differentiation in the derived state?

In the original paper, Authors state that *The G allele at rs3827760 is not present in Europeans and Africans but is seen at high frequency in East Asians and is essentially fixed in Native Americans.*
Are your results in agreement with this?

Why is it not at high frequency in the Latino sample?

----------------------------



## SNP calling(OPTIONAL for the fast )

<details>
<summary> Expand to do the optional exercise else continue with excercise 2 </summary>

![stage3](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage3.png)

We now want to estimate allele frequencies at each site without relying on genotype calls.
In other words, at each site we want to to estimate (or count) how many copies of different alleles (two in case of biallelic variants) we observe in our sample (across all sequenced individuals).
However, with low depth data direct counting of individually assigned genotypes can lead to biased allele frequencies.

ANGSD has an option to estimate **allele frequencies** taking into account data uncertainty from genotype likelihoods:


In [ ]:
# calc derived allele freq per popn
 zcat Results/EUR.EDAR.geno.gz | head

In [ ]:
angsd -doMaf

Therefore, the estimation of allele frequencies requires the specification of how to assign the major and minor alleles (if biallelic).

In [ ]:
angsd -doMajorMinor


A possible command line to estimate allele frequencies might be (this may take 1 min to run):

In [ ]:
angsd -b EUR.bams -ref $REF -out Results/EUR \
        -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
        -minMapQ 20 -minQ 20 -minInd 5 -setMinDepth 7 -setMaxDepth 30 -doCounts 1 \
        -GL 1 -doGlf 1 \
        -doMajorMinor 1 -doMaf 1

where we specify:
* -doMajorMinor 1: both alleles are inferred from genotype likelihoods
* -doMaf 1: major and minor are fixed

What are the output files?
* "Results/EUR.arg"
* "Results/EUR.mafs.gz"
`.args` file is a summary of all options used, while `.mafs.gz` file shows the allele frequencies computed at each site.

Have a look at this file which contains estimates of allele frequency values.


In [ ]:
zcat Results/EUR.mafs.gz | head

and you may see something like
```
chromo	position	major	minor	ref	knownEM	nInd
2	108999945	C	A	C	0.000004	5
2	108999946	T	A	T	0.000004	5
2	108999947	T	A	T	0.000004	5
2	108999948	C	A	C	0.000004	5
2	108999949	T	A	T	0.000004	5
2	108999950	A	C	A	0.000004	5
2	108999951	T	A	T	0.000004	5
2	108999952	G	A	G	0.000004	5
2	108999953	A	C	A	0.000002	5
```
where `knownEM` specifies the algorithm used to estimate the allele frequency which is given under that column.
Please note that this refers to the allele frequency of the allele labelled as `minor`.
The columns are: chromosome, position, major allele, minor allele, reference allele, allele frequency, p-value for SNP calling (if -SNP-pval was called), number of individuals with data.
The last column gives the number of samples with data (you can see that this never below 5 given our filtering).

You can notice that many sites have low allele frequency, probably reflecting the fact that that site is monomorphic.
We may be interested in looking at allele frequencies only for sites that are actually variable in our sample.
Therefore we want to perform a **SNP calling**.
There are two main ways to call SNPs using ANGSD with these options:
```
        -minMaf         0.000000        (Remove sites with MAF below)
        -SNP_pval       1.000000        (Remove sites with a pvalue larger)
```
Therefore we can consider assigning as SNPs sites whose estimated allele frequency is above a certain threhsold (e.g. the frequency of a singleton) or whose probability of being variable is above a specified value.



As an illustration, call SNPs by computing:
 - genotype likelihoods using GATK method;
 - major and minor alleles inferred from genotype likelihoods;
 - frequency from known major allele but unknown minor;
 - SNPs as those having MAF=>0.05.


As a general guidance, `-GL 1` ( Samtool genotype likelihood model), `-doMaf 1/2` and `-doMajorMinor 1` should be the preferred choice for genotype calling when data uncertainty is high.
If interested in analysing very low frequency SNPs, then `-doMaf 2` should be selected.
When accurate information on reference sequence or outgroup are available, one can use `-doMajorMinor` to 4 or 5.
Also, detecting variable sites based on their probability of being SNPs is generally a better choice than defining a threshold on the allele frequency.
However, various cutoffs and a dedicated filtering should be perform to assess robustness of your called SNPs.

![stage3A](https://raw.githubusercontent.com/popgenDK/courses/main/summer2023/NGSinference//stage3A.png)


Try varying the cutoff for SNP calling and record how many sites are predicted to be variable for each scenario.
Identify which sites are not predicted to be variable anymore with a more stringent cutoff (e.g. between a pair of scenario), and plot their allele frequencies.
Use the previously calculated genotype likelihoods as input file (use additonal options  -fai  and -nInd to provide information of the genome size and number of individuals).

In [ ]:
# iterate over some cutoffs (you can change these)
for PV in 0.05 1e-2 1e-4 1e-6
do
        if [ $PV == 0.05 ]; then echo SNP_pval NR_SNPs; fi
        angsd -glf Results/EUR.glf.gz -nInd 10 -fai $REF.fai -out Results/EUR.$PV \
                -doMajorMinor 1 -doMaf 1 -skipTriallelic 1 \
                -SNP_pval $PV &> /dev/null
        echo $PV `zcat Results/EUR.$PV.mafs.gz | tail -n+2 | wc -l`
done



A possible output is (your numbers may be different):
```
SNP_pval NR_SNPs
0.05 4405
1e-2 2950
1e-4 1428
1e-6 1084
```

Which sites differ from 0.05 and 0.01? What is their frequency?
This script will also print out the first 20 discordant sites (pK.EM is the p-value for the SNP calling test).



In [ ]:

mafs1 <- read.table(gzfile("~/current_folder/Results/EUR.1e-2.mafs.gz"), he=T, row.names=NULL, strings=F)
mafs5 <- read.table(gzfile("~/current_folder/Results/EUR.0.05.mafs.gz"), header=T, row.names=NULL, stringsAsFact=F)
cat("SNPs only found with p-value threshold 0.05:\n")
both <- mafs5[,2] %in% mafs1[,2]
mafs5[both,][1:20,]

hist(as.numeric(mafs5[!both,5]), main="Discordant SNPs (SNPS only found with p-value 0.05)", xlab="MAF", xlim=c(0,0.5))

hist(as.numeric(mafs5[both,][,5]), main="Concordant SNPs(SNPs found in both)", xlab="MAF", xlim=c(0,0.5))


What can you conclude from these results?
Which frequencies are more difficult to estimate and therefore affect SNP calling?



**EXERCISE 2**

Estimate derived allele frequencies for all populations of interest using a likelihood approach, without relying on genotype calls.


In [ ]:
for POP in AFR EUR EAS LAT NAM
do
        echo $POP
        angsd -b $POP.bams -ref $REF -anc $ANC -out Results/$POP.EDAR \
                -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
                -minMapQ 20 -minQ 20 -minInd 1 -setMinDepth 1 -setMaxDepth 100 -doCounts 1 \
                -GL 1 -doMajorMinor 5 -doMaf 1 -skipTriallelic 1 \
                -doGeno 3 -doPost 1 -postCutoff 0.50 \
                -r 2:109513601
done

and print the results to the screen (Column knownEM if the estimated allele frequency based on know major and minor allele)


In [ ]:

print header
paste <(echo POP) <(zcat Results/EUR.EDAR.mafs.gz  | head -n1 )
#print maf for each pop
for POP in AFR EUR EAS LAT NAM
do
    paste <(echo $POP) <(zcat Results/$POP.EDAR.mafs.gz | tail -1)
done


- What is the difference compared to what you previously estimated?
- Based on these new results is there a difference in allele frequency between East Asians (EAS) and Native amerians (NAM)?

Lets try to make a formal test. First make a file with all of the bam files for the two populations. Then make a file where the first 20 lines contains a zero ( representing the 20 NAM) and the next 20 lines contains a 1 ( representing the 20 EAS)


In [ ]:

cat NAM.bams EAS.bams > NAM_EAS.bams

cat <(yes 0 | head -n 20) <(yes 1 | head -n 20) > NAM_EAS.ybin
#view the file
echo -e "\n----NAM_EAS.ybin----\n"
cat NAM_EAS.ybin


We will use the association module in angsd to perform the test. 

In [ ]:
angsd -doAsso

We will use the frequency test (-doAsso 1) and include the "phenotype" file that we just made (NAM_EAS.ybin). 
Use the following command to test whether there is a difference between the allele frequencies of the EDAR variants in the East Asian population and the Native american population. 

In [ ]:
angsd -b NAM_EAS.bams -ref $REF -anc $ANC -out Results/$POP.EDAR \
                -uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
                -minMapQ 20 -minQ 20 -minInd 1 -setMinDepth 1 -setMaxDepth 100 -doCounts 1 \
                -GL 1 -doMajorMinor 5 -doMaf 1 -skipTriallelic 1 \
                -doGeno 3 -doPost 1 -postCutoff 0.50 \
                -r 2:109513601  -yBin NAM_EAS.ybin -doAsso 1

Lets look at the results

In [ ]:
zcat Results/NAM.EDAR.lrt0.gz

#converting Likely ratio test statistic to a p-values
Rscript -e 'cat("\nP-value is",pchisq(2.739244,1,low=F))'

- Is there a significant difference?
- Can you find other uses for such a test?

This approach can be used genome wide to detect sites with different allele frequencies accross the whole genome. The effect size can either be the difference in allele frequency or it can be measured by e.g. Fst. 
